# 1. 环境准备

## 1.1 python 环境准备

请运行以下代码完成 python 环境的安装：

In [ ]:
! pip install gradio==6.1.0 openai==2.11.0

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 大模型调用函数准备

若想了解更多关于函数部分的内容，请查阅 **2.1 函数** 部分内容。

In [ ]:
import os
from openai import OpenAI

def llm_response(content):
  client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"), 
  base_url="https://aistudio.baidu.com/llm/lmapi/v3", 
  )

  chat_completion = client.chat.completions.create(
  messages=[
    {'role': 'system', 'content': '你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'},
    {'role': 'user', 'content': content}
  ],
  model="ernie-3.5-8k",
  )

  return chat_completion.choices[0].message.content

# 2. 快速搭建 Blocks 界面

在这个示例中，我们依然使用前面定义的函数，输入是字符串类型的 name，输出则是经过处理的字符串结果。

由于 `gr.Blocks` 是一个界面容器，可以在其中灵活地添加输入组件（如 Textbox、Button）、输出组件（如 Textbox、Label）以及事件逻辑（如 click、change），因此我们通常结合 Python 的上下文管理器语法 `with ... as demo:` 来使用它。

当执行到 with 代码块时，`gr.Blocks()` 会自动完成界面的初始化，并将代码块内定义的所有组件统一收集到这个界面中。`as demo` 则是为这个界面对象命名，方便后续通过 `demo.launch()` 一键启动服务。这种写法是 `gr.Blocks` 最常见且推荐的用法，后续的代码示例也会采用这种方式来实现。

In [ ]:
import gradio as gr

def greet(name):
    return "Hello" + name + "!"

with gr.Blocks() as demo:
    name = gr.Textbox(label="Name")
    output = gr.Textbox(label="Output Box")
    greet_btn = gr.Button("Greet") # 直接写入字符串（默认是 label = "Greet"
    greet_btn.click(fn=greet, inputs=name, outputs=output)

demo.launch()

通过上面的代码，我们可以看到在 gr.Blocks 下面有三个组件，两个用于输入输出的文本框 gr.Textbox ，以及一个可点击的按钮 gr.Button。

# 3. 事件绑定方法

在上面的代码，我们看到除了之前讲到的组件以外，还有一行代码我们是我们没有见到过的，那就是 `greet_btn.click(fn=greet, inputs=name, outputs=output)`。这个代码其实我们会发现和之前 `demo = gr.Interface(fn=greet, inputs=name, outputs=output)` 是非常像的。

这个其实就是定义我们点击按钮后会触发的事件。假如我们没有定义之前，点击按钮是不会触发任何反应的。但是当我们设置了事件绑定的方法后，我们再点击按钮，就会根据我们的输入（inputs）和输出（outputs）以及触发对应的函数（fn）来执行。

当然事件绑定的组件有很多，除了 [gr.Button](https://www.gradio.app/docs/gradio/button) 里的 .click 以外，我们可以还在[Gradio 组件简介](https://www.gradio.app/docs/gradio/introduction)中找到更多触发的方法。

除此之外，有一点我们是需要注意的，就是事件绑定时所需要的组件都必须提前预设好，否则就无法成立。比如在 `greet_btn.click(fn=greet, inputs=name, outputs=output)` 写入以前，组件 `name` 和 `output` 都必须是已经创建好的了。

# 4. Blocks 布局组件

## 4.1 gr.Row 与 gr.Column

在使用 **gr.Blocks** 构建界面时，所有组件默认会 **从上到下** 依次排列。但在实际项目中，我们往往希望更灵活地控制组件的布局，比如：
- 多个输入框放在一行
- 不同的区域分栏展示  

为此，Gradio 提供了 **`gr.Row`** 和 **`gr.Column`** 两个布局组件。

### 4.1.1 gr.Column： 让组件排成一列

默认的情况下，布局的组件都是以 gr.Column() 的形式从上到下排成一列。所以即便是我们下面的代码中添加了 with gr.Column(): ，其页面也会和上面没加的是一样的。

In [ ]:
import gradio as gr

def greet(name):
    return "Hello" + name + "!"

with gr.Blocks() as demo:
    with gr.Column():
        name = gr.Textbox(label="Name")
        output = gr.Textbox(label = "Output Box")
        greet_btn = gr.Button("Greet")
        greet_btn.click(fn=greet, inputs=name, outputs=output)

demo.launch()

### 4.1.2 gr.Row：让组件在同一行排列

假如我们希望让某些组件不是从上到下依次排序，而是与别的组件共用一行，这个时候我们就可以使用 gr.Row() 将组件放到同一行。

In [ ]:
import gradio as gr

def greet(name):
    return "Hello" + name + "!"

with gr.Blocks() as demo:
    with gr.Row():
        name = gr.Textbox(label="Name")
        output = gr.Textbox(label = "Output Box")
    greet_btn = gr.Button("Greet")
    greet_btn.click(fn=greet, inputs=name, outputs=output)

demo.launch()

虽然上面的这个代码里没有添加上 gr.Column 这个列元素，但是这其实是默认存在的而已。因此这个时候即便我们下面的代码加上了 gr.Column() ，其页面展示形式是没有发生改变的。

In [ ]:
import gradio as gr

def greet(name):
    return "Hello" + name + "!"

with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            name = gr.Textbox(label="Name")
            output = gr.Textbox(label = "Output Box")
    with gr.Column():
        greet_btn = gr.Button("Greet")
        greet_btn.click(fn=greet, inputs=name, outputs=output)

demo.launch()

基于此，我们其实可以根据 gr.Column 和 gr.Row 来设计出更复杂的页面形式。

### 4.1.3 随堂训练

请使用 `gr.Blocks()` 的方法结合 `gr.Column()` 和 `gr.Row()` 为以下代码实现一个 gradio 的在线对话界面：

In [ ]:
import os
from openai import OpenAI
def llm_response(content):
    client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"), 
    base_url="https://aistudio.baidu.com/llm/lmapi/v3", 
    )

    chat_completion = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': '你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'},
        {'role': 'user', 'content': content}
    ],
    model="ernie-3.5-8k",
    )

    return chat_completion.choices[0].message.content


# TODO：gradio Blocks 界面实现代码：

## 4.2 选项卡 gr.Tab

gr.Tab 是用来创建多个可切换视图的选项卡（Tab 页）组件，每个 gr.Tab() 就是一个标签页的内容区域，你可以定义多个，用户点击不同的标签页就会显示不同的内容。

比如下面我们就定义了两个 Tab 页，第一页是关于图片生成的，第二页是关于聊天机器人的：

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    with gr.Tab("图片生成"):
        gr.Textbox(label="请输入提示词")
        gr.Image(label="生成结果")
    
    with gr.Tab("聊天机器人"):
        gr.Chatbot()
        gr.Textbox(label="请输入问题")

demo.launch()

我们在代码中其实可以看到，gr.Tab("聊天机器人") 其实这个 "聊天机器人" 是没有赋予任何参数名称的，这意味着默认情况下，`gr.Tab("聊天机器人")` 是等同于 `gr.Tab(label = "聊天机器人")` 的。除了 label 以外，我们还有一些其他的常见参数，比如：

- visible: 是否默认显示该 Tab，设为 False 时默认隐藏。
- interactive：是否能够点击该 Tab，设为 False 时无法进行点击。

## 4.3 可折叠面板 gr.Accordion

gr.Accordion 用来包裹一组组件，可以通过点击标题来展开或收起其中的内容，适合组织次要信息或可选内容。

比如下面的代码中，我们创建了一个名为 "更多设置" 的折叠面板，我们通过 open=True 或者 False 来控制其是否打开这个面板，从而显示出里面的内容，也就是 gr.Slider 和 gr.Checkbox 里的内容。

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    with gr.Accordion("更多设置", open=False): # 默认关闭
        gr.Slider(0, 100, value=50, label="音量")
        gr.Checkbox(label="启用高级模式")

    gr.Textbox(label="主要输出")

demo.launch()

当我们打开 Gradio 页面的时候，默认情况是关闭的，当我们点击小三角按钮的话，就可以打开页面显示里面的内容了。

我们在 [Accordion 的文档](https://www.gradio.app/docs/gradio/accordion) 中能够获取更多相关的参数信息。

## 4.4 统一布局组件容器 gr.group

gr.Group 是 Gradio 中的布局容器，用于将一组组件“绑定”在一起，在界面上表现为紧密排列且具有一致边距样式，也能避免组件之间被隔开太远或被打散。

gr.Group 主要是一个样式组件，使用和不使用在功能上其实差别并不大，我们可以通过下面的代码案例直观地对比一下：

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 没有使用 gr.Group: ")
    gr.Textbox(label="姓名")
    gr.Textbox(label="邮箱")
    gr.Button("提交")
    
    gr.Markdown("## 使用 gr.Group: ")
    with gr.Group():
        gr.Textbox(label="姓名")
        gr.Textbox(label="邮箱")
        gr.Button("提交")

demo.launch()

## 4.5 动态组件渲染 gr.render

[gr.render](https://www.gradio.app/docs/gradio/render) 是 Gradio 中用于动态创建组件的装饰器函数，能够根据输入状态实时生成、替换 UI 元素，实现组件的“按需渲染”。

其实际的作用如下：

- @gr.render 是一个 装饰器（decorator），用于在 gr.Blocks 中实现 动态布局。也就是说，界面上的组件、事件监听器可以根据输入或逻辑条件变化而 重构（re-render）。 
- 使用 @gr.render 后，当所指定的输入（或触发器）值发生变化时，装饰的函数会重新运行，重新生成界面组件和事件逻辑，从而让界面“动态变化”。 
- 在没有使用 gr.render 时，Blocks 中的组件和事件一旦初始定义，就相对固定；无法在运行中新增或删除组件。 @gr.render 给 Blocks 带来可变性。 

在使用 render 方法时，我们首先需要定义一个组件，并将其作为输入传递给装饰器 @gr.render()。例如，在代码中指定 inputs=input_text，表示当 input_text 的值发生变化时，show_split() 函数会被重新执行，从而触发界面的重新渲染。

另外，在装饰器下定义的 show_split() 函数，其参数会与 @gr.render 中的 inputs 一一对应。也就是说，这里的 text 参数会自动接收 input_text 当前的值，每次输入发生变化，show_split() 都会重新运行，渲染的界面内容也会随之更新。

比如，当文本框为空时，界面会显示“没有输入”的提示；而当用户开始输入文字时，程序会为每个字母动态生成一个新的 Textbox 组件，实现即时的界面更新效果。

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    input_text = gr.Textbox(label="input")

    @gr.render(inputs=input_text)
    def show_split(text):
        if len(text) == 0:
            gr.Markdown("## No Input Provided")
        else:
            for letter in text:
                gr.Textbox(letter)

demo.launch()

In [ ]:
with gr.Accordion(label="评估问题设置", open=True):
        evaluation_system_prompt = gr.Textbox(label = '系统提示词（system_prompt）', value='', info='请设置在评估模式下的系统提示词（默认为无）')
        default_evaluation_question_number = 2
        max_evaluation_question_number = 10
        default_evaluation_question_list = [
            '请给我介绍五个上海的景点',
            'Please tell me five scenic spots in Shanghai'
        ]
        evaluation_question_list = []
        with gr.Accordion(label='评估问题数量及内容',open=True):
            with gr.Row():
                with gr.Column():
                    evaluation_question_number = gr.Number(label='评估问题数', value=default_evaluation_question_number, minimum=1, maximum=max_evaluation_question_number, info='调整评估问题的数量（最多10个问题）')
                with gr.Column():
                    for i in range(max_evaluation_question_number):
                        evaluation_question_if_visible = True if i < default_evaluation_question_number else False
                        evaluation_question_value = default_evaluation_question_list[i] if i < default_evaluation_question_number else ''
                        t = gr.Textbox(label=f'评估问题{i + 1}', value=evaluation_question_value, interactive=True, placeholder=f"请输入第{i + 1}个评估的问题", visible=evaluation_question_if_visible)
                        evaluation_question_list.append(t)
        evaluation_question_number.change(evaluation_question_number_change_wrap(max_evaluation_question_number), evaluation_question_number, evaluation_question_list)

在 [@gr.render()](https://www.gradio.app/docs/gradio/render) 里，除了参数 inputs 以外，还有很有其他的参数，比如：

- triggers：除了输入值变化外（默认情况），还可以用事件（如按钮点击）来触发渲染。
- trigger_mode：决定触发渲染时的执行模式
    - 'once'：每次变化只触发一次（默认）。
    - 'multiple'：可以在上一次执行未完成时继续触发新的执行。
    - 'always_last'：只保留最新的调用，中间触发的会被跳过。

比如下面的代码中，我们添加了一个模型选择器（mode）从而决定后续显示的是文本框还是按钮，并且定义了触发器 triggers 是 input_text.submit ，也就是只有我们在 input_text 的文本框写入内容后按下回车才会触发该情况。

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    input_text = gr.Textbox(label="input")
    mode = gr.Radio(["textbox", "button"], value="textbox")

    @gr.render(inputs=[input_text, mode], triggers=[input_text.submit])
    def show_split(text, mode):
        if len(text) == 0:
            gr.Markdown("## No Input Provided")
        else:
            for letter in text:
                if mode == "textbox":
                    gr.Textbox(letter)
                else:
                    gr.Button(letter)
demo.launch()

## 4.6 状态变量 gr.State

[gr.State](https://www.gradio.app/docs/gradio/state) 是一个用于保存和传递状态值的组件，可以跨函数和事件共享数据。其实本质上就是像是 python 里的变量去保存一定的信息。

gr.State 有几个关键特点：

- 跨函数、跨事件共享：多个事件可以访问或修改这个状态。
- 持久化更新：当一个事件更新了这个状态，后续依赖这个状态的函数会拿到更新后的值。
- 界面级存储：它是 Blocks 内部的“数据存储”，不直接渲染在界面上。

在交互式页面里，如果我们想让某个数据在多次交互中保持更新，就需要用到 gr.State。

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
  text_count = gr.State(1)

  add_btn = gr.Button("Add Box")
  add_btn.click(lambda x: x + 1, text_count, text_count)

  @gr.render(inputs=text_count)
  def render_count(count):
    gr.Markdown(f"### 当前组件数：{count}")
    for i in range(count):
      gr.Textbox(label=f"Box {i + 1}")

demo.launch()

## 4.7 标识符 key

key 是每个组件的唯一标识符，告诉 Gradio：“我是上次那个组件的继续，不是新建的”。

比如下面这个代码这样，每一个文本框都会有一个 key，第一个是 textbox-0，第二个是textbox-1 这样。

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
  text_count = gr.State(1)

  add_btn = gr.Button("Add Box")
  add_btn.click(lambda x: x + 1, text_count, text_count)

  @gr.render(inputs=text_count)
  def render_count(count):
    gr.Markdown(f"### 当前组件数：{count}")
    for i in range(count):
      gr.Textbox(label=f"Box {i + 1}", key = f"textbox-{i}")

demo.launch()

这个时候即便我们写入内容后再点击按钮，内容也不会再消失了。

### 4.7.1 课堂练习
请基于上面标识符 key 的代码，不仅仅实现添加 Box 的方法，也添加删除 Box 的方法：

In [ ]:
# TODO：请将代码写在此处下方

import gradio as gr

with gr.Blocks() as demo:
  text_count = gr.State(1)

  with gr.Row():
    add_btn = gr.Button("Add Box")
    add_btn.click(lambda x: x + 1, text_count, text_count)

  @gr.render(inputs=text_count)
  def render_count(count):
    gr.Markdown(f"### 当前组件数：{count}")
    for i in range(count):
      gr.Textbox(label=f"Box {i + 1}", key = f"textbox-{i}")

demo.launch()

## 4.8 多步骤引导（Walkthrough）

gr.Walkthrough 是 Gradio 里新增的 多步骤引导（multi-step workflow） 组件，
它的作用是把交互流程拆分为多个步骤（Step），让用户可以按照步骤逐步完成操作。

在 Gradio 之前的版本里，我们有两种典型的界面构建方式：

1. 传统的左输入 + 右输出布局

- 用户在左边输入参数或上传数据，右边实时展示结果。

- 适合简单的 AI Demo 或参数调节类工具。

- 问题是：当交互步骤比较多、流程比较复杂时，所有组件堆在一个页面上，界面会显得非常拥挤，用户容易迷失。

2. 多标签页（Tabs） 或 折叠面板（Accordion）

- 可以把不同的内容分在不同的 Tab 或折叠面板中。

- 但是用户仍然可以随意跳转，没有强制的顺序控制，缺乏一个明确的“流程引导”。

所以 Walkthrough 的设计目的就是为了解决 多步骤交互 里以下几个痛点：
| 问题              | Walkthrough 的解决方式              |
| --------------- | ------------------------------ |
| **页面太复杂、组件太多**  | 每次只显示当前步骤的组件，减少视觉负担            |
| **用户不知道下一步做什么** | 按顺序逐步引导，用户操作路径被清晰分割            |
| **无法灵活控制流程**    | 通过按钮或事件控制步骤切换，支持分支或条件跳转        |
| **难以结合状态管理**    | 步骤之间可共享数据，结合 `gr.State` 实现状态流转 |

其适应的场景如下：
- 分步骤表单：注册、问卷调查、配置复杂 AI 任务
- AI 工作流：上传数据 → 设置参数 → 运行推理 → 下载结果
- 交互式教学：学生在每个步骤完成任务，逐步解锁后续操作

比如在下面的例子中，我们就实现了让用户先上传照片，再输入提示词，最后获得结果的流程。用户只有一步步按照我们的操作进行点击才能获得最终的结果。

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    with gr.Walkthrough(selected=0) as walkthrough:  # selected=0 表示初始显示第 0 步
        # 第一步：上传图片
        with gr.Step("上传图片", id=0):
            img = gr.Image()
            btn1 = gr.Button("下一步")
            btn1.click(lambda: gr.Walkthrough(selected=1), outputs=walkthrough)

        # 第二步：输入提示词
        with gr.Step("输入提示词", id=1):
            prompt = gr.Textbox()
            btn2 = gr.Button("生成结果")
            btn2.click(lambda: gr.Walkthrough(selected=2), outputs=walkthrough)

        # 第三步：查看结果
        with gr.Step("查看结果", id=2):
            gr.Markdown("这里显示生成的图片或文本")

demo.launch()

总的来说，Walkthrough 的出现，是为了让 Gradio 更好地支持：

- 分步骤、可控流程 的应用场景

- 提供比 Tabs / Accordion 更强的流程控制能力

- 减少用户在复杂应用中的认知负担，提升交互体验

### 4.8.1 课堂练习

1. 请更新下边的函数，使其能够传入系统提示词及用户提示词。
2. 使用 Walkthrough 的方法让用户逐步的输入系统提示词和用户提示词后，实现与模型进行对话。

In [ ]:
# 请先更新函数

import os
from openai import OpenAI
def llm_response(content):
    client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"), 
    base_url="https://aistudio.baidu.com/llm/lmapi/v3", 
    )

    chat_completion = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': '你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'},
        {'role': 'user', 'content': content}
    ],
    model="ernie-3.5-8k",
    )

    return chat_completion.choices[0].message.content

print(llm_response("你是谁？"))

# TODO：请实现 gradio Walkthrough 界面实现代码：